In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('./drive/MyDrive/NLP-AIBOT/DatasetSlotLabeling')

In [ ]:
df_slot_weather = pd.read_csv('./Dataset - WeatherSlot.csv', encoding = 'utf-8')
df_slot_time = pd.read_csv('./Dataset - TimeSlot.csv', encoding= 'utf-8')
df_slot_religious = pd.read_csv('./Dataset - WeatherSlot.csv', encoding= 'utf-8')
df_slot_calender = pd.read_csv('./Dataset - CalendarSlot.csv', encoding= 'utf-8')


In [ ]:
def dicts_slot_sentences(df):
    index_spilt = []
    df_nan = df.isna()
    for index in range(len(df)):
        if df_nan.loc[index, 'slot']:
            index_spilt.append(index)
            
    dicts_slot = {}
    dicts_sentences = {}
    first = 0
    for i in range(len(index_spilt)):
            dicts_slot[i] = [df.loc[index, 'slot'] for index in  range(first,index_spilt[i])]
            dicts_sentences[i] = [df.loc[index, 'word'] for index in  range(first,index_spilt[i])]
            first = index_spilt[i]+1
            
    return dicts_slot, dicts_sentences

In [ ]:
dicts_slot_weather, dicts_sentences_weather = dicts_slot_sentences(df_slot_weather)
dicts_slot_time, dicts_sentences_time = dicts_slot_sentences(df_slot_time)
dicts_slot_religious, dicts_sentences_religious = dicts_slot_sentences(df_slot_religious)
dicts_slot_calender, dicts_sentences_calender = dicts_slot_sentences(df_slot_calender)

In [ ]:
len(dicts_slot_weather)+ len(dicts_slot_time)+ len(dicts_slot_religious)+ len(dicts_slot_calender)

389

In [ ]:
for s in dicts_sentences_weather.items():
  print(s[1])
  break

['هوا', 'چطور', 'است']


In [ ]:
def create_dataset(df):
  dicts_slot, dicts_sentences = dicts_slot_sentences(df)

  slots = []
  sentences = []

  for slot, sentence in zip(dicts_slot.items(), dicts_sentences.items()):
    slots.append(slot[1]) 
    sentences.append(sentence[1])

  return slots, sentences


In [ ]:
weather_slot, weather_sentences = create_dataset(df_slot_weather)
time_slot, time_sentences = create_dataset(df_slot_time)
religious_slot, religious_sentences = create_dataset(df_slot_religious)
calender_slot, calender_sentences = create_dataset(df_slot_calender)

In [ ]:
X_dataset = [*weather_sentences, *time_sentences, *religious_sentences, *calender_sentences] #merge sentences
Y_dataset = [*weather_slot, *time_slot, *religious_slot, *calender_slot] #merge slots

In [ ]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_dataset)

In [ ]:
total_words = len(tokenizer.word_index) + 1
total_words

267

In [ ]:
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(Y_dataset)
tokenizer2.word_index

{'b-city': 3,
 'b-date': 2,
 'b-event': 9,
 'b-hour': 6,
 'b-operation': 8,
 'i-city': 10,
 'i-date': 4,
 'i-event': 7,
 'i-hour': 5,
 'i-operation': 11,
 'o': 1}

In [ ]:
seq_sentences = tokenizer.texts_to_sequences(X_dataset)
seq_slot =  tokenizer2.texts_to_sequences(Y_dataset)


In [ ]:
print(seq_sentences[10], end = ' ')
print()
print(seq_slot[10], end= ' ')

[33, 4, 2, 7, 10, 9, 1] 
[8, 1, 1, 3, 2, 1, 1] 

In [ ]:
max_sequence_len_text = max([len(x) for x in seq_sentences])
max_sequence_len_slot = max([len(x) for x in seq_slot])

In [ ]:
max_sequence_len_text, max_sequence_len_slot

(17, 17)

In [ ]:
input_sequences = np.array(pad_sequences(seq_sentences, maxlen=max_sequence_len_text, padding='post'))
target_sequences = np.array(pad_sequences(seq_slot, maxlen=max_sequence_len_slot, padding='post'))

In [ ]:
len(input_sequences), len(target_sequences)

(389, 389)

In [ ]:
n_vocab = len(tokenizer.word_index) + 1
n_classes = len(tokenizer2.word_index) + 1
n_vocab

267

In [ ]:
model = Sequential()
model.add(Embedding(n_vocab,100))
model.add(Convolution1D(128, 5, padding='same', activation='relu'))
model.add(Dropout(0.25))
model.add(LSTM(100,return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax'), ))
model.compile('rmsprop', 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
x_train = input_sequences[10:]
y_train = target_sequences[10:]

x_test = input_sequences[:10]
y_test = target_sequences[:10]



In [ ]:
y_train_t = np.eye(n_classes)[y_train]
x_train_t = x_train


In [ ]:
model.fit(x_train_t, y_train_t, epochs=100, verbose=1)

Epoch 1/100
12/12 [==============================] - 23s 32ms/step - loss: 1.9500 - accuracy: 0.4660
Epoch 2/100
12/12 [==============================] - 0s 32ms/step - loss: 1.0243 - accuracy: 0.7229
Epoch 3/100
12/12 [==============================] - 0s 31ms/step - loss: 0.8424 - accuracy: 0.7340
Epoch 4/100
12/12 [==============================] - 0s 30ms/step - loss: 0.6998 - accuracy: 0.7582
Epoch 5/100
12/12 [==============================] - 0s 32ms/step - loss: 0.5259 - accuracy: 0.8313
Epoch 6/100
12/12 [==============================] - 0s 30ms/step - loss: 0.4116 - accuracy: 0.8739
Epoch 7/100
12/12 [==============================] - 0s 30ms/step - loss: 0.3265 - accuracy: 0.9089
Epoch 8/100
12/12 [==============================] - 0s 29ms/step - loss: 0.2269 - accuracy: 0.9411
Epoch 9/100
12/12 [==============================] - 0s 30ms/step - loss: 0.1762 - accuracy: 0.9476
Epoch 10/100
12/12 [==============================] - 0s 30ms/step - loss: 0.1484 - accuracy: 0.957

In [ ]:
import progressbar
accuracy = tf.keras.metrics.CategoricalAccuracy()
n_epochs = 100

for i in range(n_epochs):
    print("Training epoch {}".format(i))
    
    bar = progressbar.ProgressBar(max_value=len(x_train))
    for n_batch, sent in bar(enumerate(x_train)):
        label = y_train[n_batch]
        # Make labels one hot
        label = np.eye(n_classes)[label][np.newaxis,:] 
        # View each sentence as a batch
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #ignore 1 word sentences
            model.train_on_batch(sent, label)
            #logits = model(sent)
            #accuracy.update_state(sent, logits)
        #print("accuracy : ", accuracy.result())

  2% (10 of 379) |                       | Elapsed Time: 0:00:00 ETA:   0:00:06

Training epoch 0


100% (379 of 379) |######################| Elapsed Time: 0:00:06 Time:  0:00:06
  2% (10 of 379) |                       | Elapsed Time: 0:00:00 ETA:   0:00:05

Training epoch 1


100% (379 of 379) |######################| Elapsed Time: 0:00:06 Time:  0:00:06
  2% (10 of 379) |                       | Elapsed Time: 0:00:00 ETA:   0:00:05

...

100% (379 of 379) |######################| Elapsed Time: 0:00:06 Time:  0:00:06
  2% (10 of 379) |                       | Elapsed Time: 0:00:00 ETA:   0:00:07

Training epoch 98


100% (379 of 379) |######################| Elapsed Time: 0:00:06 Time:  0:00:06
  2% (10 of 379) |                       | Elapsed Time: 0:00:00 ETA:   0:00:07

Training epoch 99


100% (379 of 379) |######################| Elapsed Time: 0:00:06 Time:  0:00:06


In [ ]:
def test_string(text):
    
    string = text
    sent = tokenizer.texts_to_sequences([string.split()])
    input_test = np.array(pad_sequences(sent, maxlen=max_sequence_len_text, padding='post'))
    
    pred = model.predict(input_test)
    pr = np.argmax(pred,-1)[0]
    len_string = len(string.split())
    seq_slot_predict = pr[:len_string]
    
    slot_index = tokenizer2.word_index
    
    revers_slot_index = {value:key for key, value in slot_index.items()}
    revers_slot_index[0] = 'o'
    
    slot_text = [revers_slot_index[index_seq] for index_seq in seq_slot_predict]
    
    dicts_final = {}
    for (word, slot) in zip(string.split(), slot_text):
        dicts_final[word] = slot
    
    return string.split(), slot_text, dicts_final
    

In [ ]:
list(x_test[0])

[38, 71, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
string = "دمای هوای تهران امروز چند درجه است"
split_text, slot_text, dicts_text = test_string(string)
dicts_text


{'است': 'i-date',
 'امروز': 'i-hour',
 'تهران': 'b-date',
 'درجه': 'i-date',
 'دمای': 'b-hour',
 'هوای': 'b-event',
 'چند': 'i-date'}

In [ ]:
y_test_val = np.eye(n_classes)[y_test]
x_test_val = x_test


In [ ]:
x_test_val.shape, y_test_val.shape

((10, 17), (10, 17, 12))

In [ ]:
model.evaluate(x_test_val, y_test_val)

1/1 [==============================] - 1s 1s/step - loss: 1.9144e-07 - accuracy: 1.0000


[1.9143617180361616e-07, 1.0]

In [ ]:
model.save_weights('model.h5')

In [ ]:
model.save('modelslot.h5')

TypeError: ignored

In [ ]:
 tf.keras.models.save_model( model, 'model', overwrite=True, include_optimizer=True )

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [ ]:
import pickle

In [ ]:
!python --version

Python 3.7.11
